<img src="https://gitlab.version.fz-juelich.de/grosch1/pvlink_demo_for_jupytercon/-/raw/JupyterCon2020/img/logo.png"
     style="float:right; width:25%; height:25%; margin-top:25px;">

# Jupyter for interactive in-situ visualization with ParaView/Catalyst

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.version.fz-juelich.de%2Fgrosch1%2Fpvlink_demo_for_jupytercon/JupyterCon2020)  
Try out the *pvlink* extension and the example notebooks in binder.

# Large dataset with a parallel `pvserver`

Start a parallel ParaView server on the command line, for example via `$ mpirun -np <numberOfProc> ./pvserver`.

In this example, 64 compute nodes were used for a dataset of approximatly 1.2TB.


In [1]:
import ipywidgets as widgets
import os
import pvlink

from paraview.simple import *

## Setup pvlink renderer
We are using *jupyter server proxy* to access our ParaViewWeb server since it is running on the HPC system.

In [2]:
renderer = pvlink.RemoteRenderer(
    baseURL='jupyter-jsc.fz-juelich.de'+os.environ['JUPYTERHUB_SERVICE_PREFIX'], 
    disableExternalPort=True, useJupyterServerProxyHttps=True,
    pvserverHost="localhost", pvserverPort=11111,
    timeout="0"
)

## Setup paraview pipeline

### Setup RenderView

In [3]:
renderView = CreateView('RenderView', 'view')
pvlink.utility.SetRecommendedRenderSettings(renderView)

renderView.Background = [0, 0, 0]
renderView.OrientationAxesVisibility = 0
renderView.ShowAnnotation = 1

### Read .xmf file

In [4]:
box2xmf = XDMFReader(FileNames=['./visu/box2c.xmf'])
box2xmf.GridStatus = ['Structured Grid']
box2xmf.UpdatePipeline()

In [5]:
# box2xmfDisplay = Show(box2xmf, renderView)
# box2xmfDisplay.SetRepresentationType('Volume')

In [6]:
# Hide(box2xmf, renderView)

### Apply filters

In [7]:
contour = Contour(Input=box2xmf)
contour.Isosurfaces = [0.3]

In [8]:
contourDisplay = Show(contour, renderView, 'GeometryRepresentation')
contourDisplay.SetRepresentationType('Surface')
contourDisplay.RescaleTransferFunctionToDataRange(True, False)
contourDisplay.SetScalarBarVisibility(renderView, True);

In [9]:
ColorBy(contourDisplay, ('POINTS', 'PS'))
psLUT = GetColorTransferFunction('PS')
psLUT.ApplyPreset('Blue Orange (divergent)', True);

### Reset camera and focal point and display renderer

In [10]:
pvlink.utility.ResetCamera(renderView, renderer)
renderer.update_render()

In [16]:
renderContainer = widgets.Box(children=[renderer], layout={'height': '800px'})
display(renderContainer)

Box(children=(RemoteRenderer(authKey='1da0ea24ad20daada608fc5de87325fb709b07fb8816cb7e', sessionURL='wss://jup…

## Print memory usage information

In [12]:
dataInfo = box2xmf.GetDataInformation()

memory_size = dataInfo.DataInformation.GetMemorySize()
number_points = dataInfo.DataInformation.GetNumberOfPoints()
number_cells = dataInfo.DataInformation.GetNumberOfCells()

In [13]:
from paraview import benchmark
memuse = benchmark.logbase.get_memuse()

In [14]:
print("Number of points:\t", number_points)
print("Number of cells:\t", number_cells)
print("Memory size:\t\t", memory_size)

Number of points:	 77976270099
Number of cells:	 77629109375
Memory size:		 1218379428


In [15]:
for use in memuse:
    print(use)

CL[0] 252244 / 52058712
DS_RS[0] 9798604 / 52058712
DS_RS[1] 9676520 / 52058712
DS_RS[2] 9675744 / 52058712
DS_RS[3] 9698376 / 52058712
DS_RS[4] 9674232 / 51420436
DS_RS[5] 9698396 / 51420436
DS_RS[6] 9698368 / 51420420
DS_RS[7] 9720144 / 51420420
DS_RS[8] 9677052 / 49780008
DS_RS[9] 9698408 / 49780008
DS_RS[10] 9698244 / 49780008
DS_RS[11] 9720116 / 49780008
DS_RS[12] 9677580 / 52932288
DS_RS[13] 9698372 / 52932288
DS_RS[14] 10184256 / 52932288
DS_RS[15] 9720208 / 52932288
DS_RS[16] 9675208 / 51310944
DS_RS[17] 9698412 / 51310944
DS_RS[18] 9698252 / 51310944
DS_RS[19] 9720132 / 51310944
DS_RS[20] 9676176 / 50058232
DS_RS[21] 9698400 / 50058232
DS_RS[22] 9698252 / 50058232
DS_RS[23] 9720176 / 50058232
DS_RS[24] 9698896 / 51642404
DS_RS[25] 9719724 / 51642404
DS_RS[26] 9694356 / 51642404
DS_RS[27] 9717364 / 51642404
DS_RS[28] 9696288 / 49589292
DS_RS[29] 9717012 / 49589292
DS_RS[30] 9698384 / 49589292
DS_RS[31] 9720068 / 49589292
DS_RS[32] 9674236 / 51590984
DS_RS[33] 9697320 / 51590984

In [ ]:
# Disconnect()